# Practice №11
## by Anton Kondrashov
##### HSE, CS, BSE 141(1)

### Dependencies

In [1]:
import sklearn
import pandas as pd
import numpy as np
import codecs
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestRegressor

### Datasets

In [2]:
def unpack_dataset(path, encoding='utf-8'):
    dicts = []
    with codecs.open(path, 'r', encoding=encoding) as f:
        for line in f:
            dicts.append(eval(line))
    
    return pd.DataFrame(dicts)

In [3]:
#movie_descriptions = unpack_dataset('/Users/kondranton/Desktop/Datasets/movie_descriptions.txt')
#test_feedback = unpack_dataset('/Users/kondranton/Desktop/Datasets/test_feedback.txt', encoding='ISO-8859-1')
train_feedback = unpack_dataset('/Users/kirillcherepanov/Downloads/train_feedback.txt', encoding='ISO-8859-1')
#test_answers = pd.read_csv('/Users/kondranton/Desktop/Datasets/test_sample_answers.txt')

### Get matricies

In [4]:
vectorizer = TfidfVectorizer(min_df=0.001, stop_words='english')
text_matrix = pd.DataFrame(vectorizer.fit_transform(train_feedback['text']).toarray())

In [5]:
#future_use
# train_feedback.drop(['text','summary', 'profileName', 'time'], axis=1, inplace=True)

In [6]:
#some code is suitable for future use
train_feedback = pd.concat([train_feedback['score'], text_matrix], axis=1)

train_percentage = 50000/len(train_feedback)
mask = np.random.rand(len(train_feedback)) < train_percentage
train_set = train_feedback[mask]
test_set = train_feedback[~mask]

In [7]:
train_x = train_set[[x for x in list(train_set.columns) if x != 'score']]
train_y = train_set[['score']].as_matrix()
test_x = test_set[[x for x in list(train_set.columns) if x != 'score']]
test_y = test_set[['score']].as_matrix()

In [8]:
regressor = RandomForestRegressor(n_jobs=-1, **{'max_features': 0.07, 'min_samples_leaf': 2})
regressor.fit(train_x, [x[0] for x in train_y])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=0.07, max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=2,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=-1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [9]:
regressor.score(test_x, [x[0] for x in test_y])

0.41468425752437954

In [11]:
regressor.feature_importances_[:9]

array([  4.71566262e-05,   2.78644785e-06,   0.00000000e+00,
         4.80993097e-04,   2.06751336e-04,   1.07450118e-05,
         0.00000000e+00,   1.23888952e-05,   6.40116658e-05])

In [18]:
importances = pd.concat([pd.DataFrame(train_x.columns), pd.DataFrame(regressor.feature_importances_)], axis=1)

In [30]:
importances.columns = ['feat', 'import']
important_indexes = importances.sort_values('import', ascending=False)[:9]['feat'].as_matrix()

In [31]:
names = vectorizer.get_feature_names()
[names[i] for i in important_indexes]

['worst',
 'waste',
 'bad',
 'great',
 'boring',
 'best',
 'stupid',
 'terrible',
 'money']

### Conclusion

As we can see, these words have the most significant impact on a review score. The words express someone's opinion, so they can be intepreted as a movie empirical score. 